In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/yutingchen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/yutingchen/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
import os
import uuid
import base64
from IPython import display
from unstructured.partition.pdf import partition_pdf
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.schema.messages import HumanMessage, SystemMessage
from langchain.schema.document import Document
from langchain.vectorstores import FAISS
from langchain.retrievers.multi_vector import MultiVectorRetriever
from openai import OpenAIError

os.environ['OPENAI_API_KEY'] = "sk-****"
output_path = "./images"

In [7]:
from PyPDF2 import PdfReader

# 指定目錄路徑
directory = 'Glossary of racism concepts'

# 獲取該目錄下所有 PDF 檔案的清單
pdf_files = [f for f in os.listdir(directory) if f.endswith('.pdf')]

raw_pdf_elements = []

# 迭代每個 PDF 檔案並讀取內容
for pdf_file in pdf_files:
    pdf_path = os.path.join(directory, pdf_file)
    reader = PdfReader(pdf_path)
    
    # 將每頁的內容讀取到 raw_pdf_elements 中
    for page_num in range(len(reader.pages)):
        page = reader.pages[page_num]
        raw_pdf_elements.append(page.extract_text())

#print(raw_pdf_elements)


In [15]:
# 假設 summary_chain 是你用來生成摘要的函數
text_elements = []
text_summaries = []
table_summaries = []
table_elements = []

summary_prompt = """
Summarize the following {element_type}: 
{element}
"""
summary_chain = LLMChain(
    llm=ChatOpenAI(model="gpt-3.5-turbo", max_tokens=1024), 
    prompt=PromptTemplate.from_template(summary_prompt)
)

for e in raw_pdf_elements:
    if isinstance(e, str):  # 檢查是否為文本元素
        text_elements.append(e)
        summary = summary_chain.run({'element_type': 'text', 'element': e})
        text_summaries.append(summary)
    elif 'Table' in repr(e):  # 檢查是否為表格元素
        table_elements.append(e)
        summary = summary_chain.run({'element_type': 'table', 'element': e})
        table_summaries.append(summary)

# 檢查讀取到的內容

#print(text_summaries)
#print(table_summaries)
#print(table_elements)

In [10]:
# Get image summaries
image_elements = []
image_summaries = []

def encode_image(image_path):
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode('utf-8')
    
def summarize_image(encoded_image):
    prompt = [
        SystemMessage(content="You are a bot that is good at analyzing images."),
        HumanMessage(content=[
            {
                "type": "text", 
                "text": "Describe the contents of this image."
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{encoded_image}"
                },
            },
        ])
    ]
    response = ChatOpenAI(model="gpt-4-turbo", max_tokens=1024).invoke(prompt)
    return response.content
    
for i in os.listdir(output_path):
    if i.endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(output_path, i)
        encoded_image = encode_image(image_path)
        image_elements.append(encoded_image)
        summary = summarize_image(encoded_image)
        image_summaries.append(summary)

In [16]:
# Create Documents and Vectorstore
documents = []
retrieve_contents = []

for e, s in zip(text_elements, text_summaries):
    i = str(uuid.uuid4())
    doc = Document(
        page_content = s,
        metadata = {
            'id': i,
            'type': 'text',
            'original_content': e
        }
    )
    retrieve_contents.append((i, e))
    documents.append(doc)
    
for e, s in zip(table_elements, table_summaries):
    doc = Document(
        page_content = s,
        metadata = {
            'id': i,
            'type': 'table',
            'original_content': e
        }
    )
    retrieve_contents.append((i, e))
    documents.append(doc)
    
for e, s in zip(image_elements, image_summaries):
    doc = Document(
        page_content = s,
        metadata = {
            'id': i,
            'type': 'image',
            'original_content': e
        }
    )
    retrieve_contents.append((i, s))
    documents.append(doc)

vectorstore = FAISS.from_documents(documents=documents, embedding=OpenAIEmbeddings())

C:\Users\Li Cheng Shuan\AppData\Roaming\Python\Python312\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [17]:
answer_template = """
Answer the question based only on the following context, which can include text, images and tables:
{context}
Question: {question} 
"""
answer_chain = LLMChain(llm=ChatOpenAI(model="gpt-3.5-turbo", max_tokens=1024), prompt=PromptTemplate.from_template(answer_template))

def answer(question):
    relevant_docs = vectorstore.similarity_search(question)
    context = ""
    relevant_images = []
    for d in relevant_docs:
        if d.metadata['type'] == 'text':
            context += '[text]' + d.metadata['original_content']
        elif d.metadata['type'] == 'table':
            context += '[table]' + d.metadata['original_content']
        elif d.metadata['type'] == 'image':
            context += '[image]' + d.page_content
            relevant_images.append(d.metadata['original_content'])
    result = answer_chain.run({'context': context, 'question': question})
    return result, relevant_images

In [18]:
import base64
from IPython import display
result, relevant_images = answer("What is Deviant?")
print(result)
for e in relevant_images:
    display.display(display.Image(base64.b64decode(e)))


Deviant is a term used to characterize Asian Americans as immoral and dangerous in one instance and meek and robotic in another. It is one of the subcategories used in the Committee of 100's Glossary of Anti-Asian Terms and Tropes.
